In [264]:
import os
import importlib
import logging
importlib.reload(logging)
import framework
importlib.reload(framework)
import bert_ner
importlib.reload(bert_ner)
import infer_bert_classifier
importlib.reload(infer_bert_classifier)
import bert_utils
importlib.reload(bert_utils)
import pandas as pd
from framework import DataCuration, FeatureEngineering
from bert_ner import TaskNER, FeatureEngineeringNER, BERTNER

# Define some constants and configurations
logging.getLogger().setLevel(logging.INFO)
ACCESS_TOKEN = 'WUpGevbWC9lsnTW8quNUtmWRdAEM89'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ahsaasbajaj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Set up the task details. This notebook handles NER (for labeling person and company names)

In [265]:
DATASET = 'w2' # supports w2 and resume

# also supports 'address' as 4th category in task
TASK_CONFIG = {
    'task': 'ner',
    'num_labels': 3,
    'labels_dict': {'person' : 0, 'org' : 1, 'none': 2}
}

task = TaskNER(TASK_CONFIG)

Set paths for datasets and goldens (local or ib, both work).
Specify configurations

In [266]:
W2_DATA = [
   '/Users/ahsaasbajaj/Documents/Data/W2/W2_Clean_DataSet_01_20Sep2019/ocr/out-pdf'
]
W2_GOLDEN = [
   '/Users/ahsaasbajaj/Documents/Data/W2/W2_Clean_DataSet_01_20Sep2019/golden/goldens.csv'
]

GOLDEN_CONFIG = {
    'path': W2_GOLDEN,
    'is_local': True,
    'index_field_name':'File_BaseName',
    'file_type': 'csv',
    'identifier': 'file'
}
DATASET_CONFIG = {
    'path': W2_DATA,
    'is_local': True, 
    'file_type': 'ibocr',
    'identifier': lambda path: os.path.basename(path).split('.ibocr')[0].split('.pdf')[0],
    'convert2txt': True
}

data = DataCuration(ACCESS_TOKEN, DATASET_CONFIG, GOLDEN_CONFIG)

INFO:root:Loading dataset from /Users/ahsaasbajaj/Documents/Data/W2/W2_Clean_DataSet_01_20Sep2019/ocr/out-pdf
INFO:root:1994 files loaded
INFO:root:Loading goldens from /Users/ahsaasbajaj/Documents/Data/W2/W2_Clean_DataSet_01_20Sep2019/golden/goldens.csv
INFO:root:Total files Goldens: (2000, 45)
INFO:root:Total files found in the source with unique index: (1994, 45)
INFO:root:Processing 1994 IBOCR files to txt


In [267]:
assert(len(data.dataset.keys()) == data.golden.shape[0])

In [268]:
PROCESSING_CONFIG = {
    'X_DIST_THRESHOLD': 200,
    'RANDOM_SEQ_LEN': 5
}

# use as negative filter to generate random sequences
employer_address = ['Employer\'s Street Address', 'Employer\'s City-State-Zip']
employee_address = ['Employee Street Address', 'Employee\'s City-State-Zip']
other_location = ['Locality Name_1', 'State_1', 'Locality Name_2', 'State_2']

DATA_ARGS = {
    'task': task,
    'dataset': data,
    'candidates_fields': {
        'org':'Employer\'s Name',
        'person':'Employee Name',
        'address1': employer_address,
        'address2': employee_address,
        'address3': other_location
    },
    'use_random_seq': True, # if False, then use phrases generated by generate_candidates_phrases() for 'None' class in classifier training data
    'mode': 'person-org' # person-org or person-org-address, need to change label_dict in Task accordingly
}

data.generate_candidates_phrases(PROCESSING_CONFIG)
data.compare_candidates_and_goldens(DATA_ARGS['candidates_fields'])

INFO:root:Generating candidates for 1994 files
INFO:root:For X_DIST_THRESHOLD configuraion: 200
INFO:root:total files: 1994
person names found in candidates: 1987
org names found in candidates: 1992



In [269]:
data.split_train_test()

INFO:root:Total samples 1994, training samples: 1413, Test Samples: 581
INFO:root:Total samples 1994, training samples: 1395, Test Samples: 599


Generate test data from goldens (from actual persons and company names) or from ibocr (using candidate phrases generated by processIBOCR2candidatePhrases())

In [270]:
fe = FeatureEngineeringNER(DATA_ARGS)

In [272]:
train_data, test_data = fe.create_train_test_data(PROCESSING_CONFIG)
train_data.head(20)

INFO:root:Generating labeled data for dataset with shape (1413, 45)
INFO:root:Cleaned text size 1413
INFO:root:Random Sequence text size 1413
INFO:root:Mode person-org
INFO:root:Generating labeled data for dataset with shape (581, 45)
INFO:root:Cleaned text size 581
INFO:root:Random Sequence text size 581
INFO:root:Mode person-org
INFO:root:train data: (4239, 2), test test: (1743, 2)


,context,label
1109,Justin Hernandez,0
1080,furnished Internal Revenue If required,2
347,Alexander Group Group,1
875,"Lynch, Garcia and Pham Inc",1
217,information furnished Internal Revenue Cut,2
87,Combs-Smith Ltd,1
1332,Jones Ltd Inc,1
791,Visit IRS Website STATEMENT OMB,2
1294,"Little, Rodriguez and White Group",1
93,Wilson-Atkins and Sons,1
